In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
# Load the dataset with the correct encoding
df = pd.read_csv('/content/Super_Store_data.csv', encoding='ISO-8859-1')
df.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
3,6,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600,7,0.00,14.1694
4,11,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,FUR-TA-10001539,Furniture,Tables,Chromcraft Rectangular Conference Tables,1706.1840,9,0.20,85.3092


In [4]:
#data preprocessing
#Handling missing data (if any)
df.fillna(method='ffill',inplace = True)

<ipython-input-4-0ff90d467154>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill',inplace = True)


In [5]:
#feature Engineering
#Extracting date components
df['Order Year'] = pd.to_datetime(df['Order Date']).dt.year
df['Order Month'] = pd.to_datetime(df['Order Date']).dt.month
df['Order Day'] = pd.to_datetime(df['Order Date']).dt.day

In [6]:
#Selecting Features and Target Variables
features = ['Ship Mode', 'Segment', 'Country', 'City', 'State', 'Region', 'Category', 'Sub-Category',
            'Sales', 'Quantity', 'Discount', 'Profit', 'Order Year', 'Order Month', 'Order Day']
target = 'Sales'

X = df[features]
y = df[target]

In [7]:
# Splitting the data to training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X ,y, test_size=0.2, random_state = 42)

In [8]:
# Preprocessing the pipeline for categorical and numerical values
numerical_features = ['Sales', 'Quantity', 'Discount', 'Profit', 'Order Year', 'Order Month', 'Order Day']
categorical_features = ['Ship Mode', 'Segment', 'Country', 'City', 'State', 'Region', 'Category', 'Sub-Category']

numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer,categorical_features)
    ])


In [9]:
# Model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

In [11]:
# Hyperparameter tuning using GridSearchCV
param_grid = {
    'regressor__n_estimators': [100, 200],
    'regressor__max_depth': [10, 20, None],
    'regressor__min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Sales',
                                                                          'Quantity',
                                                                          'Discount',
                                                                          'Profit',
                                                                          'Order '
                                                                          'Year',
                                                                          'Order '
                                                                          'Month',
                                                                          'Order '
                                                                          'Day']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Ship '
                                                                          'Mode',
                                                                          'Segment',
                                                                          'Country',
                                                                          'City',
                                                                          'State',
                                                                          'Region',
                                                                          'Category',
                                                                          'Sub-Category'])])),
                                       ('regressor',
                                        RandomForestRegressor(random_state=42))]),
             param_grid={'regressor__max_depth': [10, 20, None],
                         'regressor__min_samples_split': [2, 5, 10],
                         'regressor__n_estimators': [100, 200]},
             scoring='neg_mean_squared_error')

In [12]:
#best Model
best_model = grid_search.best_estimator_

#Prediction
y_pred = best_model.predict(X_test)

#evaluate the model
mse = mean_squared_error(y_test,y_pred)
r2 = r2_score(y_test,y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R2 Score: {r2}')

Mean Squared Error: 561.2411006017929
R2 Score: 0.9981683937290597


In [18]:
#feature importance
feature_importances = best_model.named_steps['regressor'].feature_importances_
feature_names = numerical_features + list(grid_search.best_estimator_.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(categorical_features))

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending = False)

print(importance_df)


                Feature  Importance
0                 Sales    0.998194
4            Order Year    0.000364
399         Region_East    0.000149
6             Order Day    0.000143
3                Profit    0.000141
..                  ...         ...
177       City_Longview    0.000000
318         City_Tigard    0.000000
317  City_Thousand Oaks    0.000000
246    City_Perth Amboy    0.000000
319    City_Tinley Park    0.000000

[407 rows x 2 columns]


In [19]:
#cross validation scores
cv_results = pd.DataFrame(grid_search.cv_results_)
print(cv_results[['mean_test_score','params']])

    mean_test_score                                             params
0       -938.345975  {'regressor__max_depth': 10, 'regressor__min_s...
1       -921.208633  {'regressor__max_depth': 10, 'regressor__min_s...
2      -1204.772995  {'regressor__max_depth': 10, 'regressor__min_s...
3      -1210.038286  {'regressor__max_depth': 10, 'regressor__min_s...
4      -1954.666265  {'regressor__max_depth': 10, 'regressor__min_s...
5      -1881.301929  {'regressor__max_depth': 10, 'regressor__min_s...
6       -966.872960  {'regressor__max_depth': 20, 'regressor__min_s...
7       -985.533758  {'regressor__max_depth': 20, 'regressor__min_s...
8      -1172.588808  {'regressor__max_depth': 20, 'regressor__min_s...
9      -1210.162036  {'regressor__max_depth': 20, 'regressor__min_s...
10     -1979.789452  {'regressor__max_depth': 20, 'regressor__min_s...
11     -1898.200851  {'regressor__max_depth': 20, 'regressor__min_s...
12      -966.872960  {'regressor__max_depth': None, 'regressor__min...
13    

In [23]:
# Saving the model
import joblib
joblib.dump(best_model, 'sales_forecasting_model.pkl')
# Save the preprocessor after fitting
joblib.dump(preprocessor, 'preprocessor.pkl')


['preprocessor.pkl']

In [33]:
# Save the entire pipeline (preprocessor + model)
joblib.dump(best_model, 'sales_forecasting_pipeline.pkl')

# Make predictions on new data
def predict_new_data(new_data):
    # Use the saved pipeline to make predictions
    predictions = best_model.predict(new_data)
    return predictions

# Load the saved pipeline for future use
best_model = joblib.load('sales_forecasting_pipeline.pkl')

# Example new data (ensure it has the same features as the original training data)
new_data = pd.DataFrame({
    'Ship Mode': ['Second Class', 'First Class'],
    'Segment': ['Consumer', 'Corporate'],
    'Country': ['United States', 'United States'],
    'City': ['New York', 'Los Angeles'],
    'State': ['New York', 'California'],
    'Region': ['East', 'West'],
    'Category': ['Furniture', 'Office Supplies'],
    'Sub-Category': ['Chairs', 'Binders'],
    'Sales': [500.0, 200.0],
    'Quantity': [3, 2],
    'Discount': [0.1, 0.2],
    'Profit': [50.0, 20.0],
    'Order Year': [2023, 2023],
    'Order Month': [8, 8],
    'Order Day': [15, 16]
})

# Make predictions on the new data
predictions = predict_new_data(new_data)

# Output the predictions
print(predictions)

[498.63211308 199.94499215]
